In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import os
import socket
import cv2
import scipy
from sklearn.model_selection import GridSearchCV

import PIL
from PIL import Image

import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# get data from folder
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    size = 80, 80, 3
    X = []
    y = []
    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
            if wbc_type in ['NEUTROPHIL']:
                label = 0
            elif wbc_type in ['EOSINOPHIL']:
                label = 1
            elif wbc_type in ['MONOCYTE']:
                label = 2
            elif wbc_type in ['LYMPHOCYTE']:
                label = 3

            for image_filename in os.listdir(folder + wbc_type):
                img_file = Image.open(folder + wbc_type + '/' + image_filename)
                if img_file is not None:
                    img_file.thumbnail(size)
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)

    X = np.asarray(X)
    print(X.shape)
    y = np.asarray(y)
    nb_classes = 4 # number of categories
    targets = y.reshape(-1)
    yy = np.eye(nb_classes)[targets]
    return X,yy,y

# get data
X_train, y_train_nn,y_train = get_data('train//')
X_test, y_test_nn,y_test = get_data('test//')
X_val, y_val_nn,y_val = get_data('validation//')

def convert(x):
    nsamples, nx, ny,dim = x.shape
    return (x.reshape((nsamples,nx*ny*dim)),x.reshape((nsamples,nx*ny,dim)))
X_train_new=convert(X_train)[0]
X_val_new=convert(X_val)[0]
X_test_new = convert(X_test)[0]

y_trainHot_new=y_train
y_valHot_new=y_val
y_testHot_new = y_test


(1000, 60, 80, 3)
(55, 60, 80, 3)
(55, 60, 80, 3)


In [ ]:
# XGboost
# define parameter
n_estimators = [50, 100, 150]
max_depth = [2, 4, 6, 8]
learning_rate = [0.001, 0.01, 0.05, 0.1]
model = xgb.XGBClassifier()

# start tuning
for depth in max_depth:
    for n in n_estimators:
        for lr in learning_rate:
            print("depth:" ,depth)
            print("n_estimator:" ,n)
            print("learn rate:" ,lr)
            
            # other parameter are based on kaggle report 
            model = xgb.XGBClassifier(objective="binary:logistic", random_state=42,
                             colsample_bytree = 0.7072947899294361,
                             gamma = 0.3227361479535839,
                             learning_rate = lr,
                             max_depth = depth,
                             n_estimators = n,
                             subsample = 0.9815714308010349, verbose=1)
            # predict on validation set
            model.fit(X_train_new, y_train)
            pred = model.predict(X_val_new)
            cm=accuracy_score(y_val, pred)
            print('Accuracy: ',cm) 


In [7]:
# fit model based on best parameter performance
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42,
                             colsample_bytree = 0.7072947899294361,
                             gamma = 0.3227361479535839,
                             learning_rate = 0.1,
                             max_depth = 8,
                             n_estimators = 100,
                             subsample = 0.9815714308010349, verbose=1)
# predict on test data 
model.fit(X_train_new, y_train)
pred = model.predict(X_test_new)
cm=accuracy_score(y_test, pred)
print('Accuracy: ',cm) 

Accuracy:  0.5272727272727272
